# CK0223 - Mineração de Dados

## Lista 02 - Extração de Dados

### Dados do discente:
**Nome**: Luiza Esther Martins Pessoa
**Matrícula**: 555516

### Vídeo Youtube:
[Mineração de Dados: Lista 02 - Extração de Dados (Explicando o código)](https://youtu.be/G3_uYOBh5VU)

### GitHub:
[EstherMart - Extração de Dados](https://github.com/EstherMart/Data-Mining/blob/main/Lista02_ExtracaoDeDados/extracao.ipynb)

---

### **(a)** Ler o dataset *fakeTelegram.BR_2022.csv*

Como realizado na [Lista 01 - Tratamento de Dados](https://github.com/EstherMart/Data-Mining/blob/main/Lista01_TratamentoDeDados/tratamento.ipynb), ler o dataset é o primeiro passo para iniciarmos a extraçãa, manipulação e tratamento dos dados. 

Para fazer isso, começamos importando as bibliotecas necessárias para leitura (`pandas`) e para download local da base de dados (`gdown`). Além disso, vale ressaltar que existem outras formas de realizar o upload para o repositório local, mas decidi seguir a lógica de puxar e realizar o download de base utilizando apenas o link disponibilizado pelo professor.

**Importante**: Alguns trechos de código serão reutilizados, visto que são as mesmas exigências em ambas as listas e tal solicitação já foi resolvida anteriormente.

In [2]:
# IMPORTAÇÃO DE BIBLIOTECAS
import gdown
import pandas as pd
import requests
from io import StringIO

Fazendo o download do dataset para o repositório local

In [ ]:
origem_url = 'https://drive.google.com/file/d/1c_hLzk85pYw-huHSnFYZM_gn-dUsYRDm/view'

# O ID do arquivo (necessário para fazer o download direto) está entre os últimos elementos da URL.
# Fazemos um split na URL usando '/' como separador e pegamos o penúltimo elemento da lista.
# Isso funciona porque a estrutura da URL é:
# https://drive.google.com/file/d/ID_DO_ARQUIVO/view
# E ao aplicar url.split('/'), o resultado será:
# ['https:', '', 'drive.google.com', 'file', 'd', 'ID_DO_ARQUIVO', 'view?...']
# Portanto, o ID está na posição -2 (penúltima).

file_id = origem_url.split('/')[-2]

# URL do arquivo no formato aceito pelo gdown
url = f'https://drive.google.com/uc?id={file_id}'

# Nome local do arquivo que será baixado
output = 'fakeTelegram.BR_2022.csv'

# Baixando o arquivo com gdown
gdown.download(url, output, quiet=False)

Leitura do dataset utilizando o método `.read_csv()`, pois permite carregar dados estruturados a partir de arquivos *CSV* para um DataFrame do pandas, sendo uma estrutura tabular extremamente versátil para análise de dados e visualmente intuitiva.

In [3]:
df_inicial = pd.read_csv("fakeTelegram.BR_2022.csv")

Os códigos abaixo foram implementados apenas para fins de **visualização e verificação inicial dos dados carregados**, com o objetivo de:
- Obter uma compreensão rápida da dimensão do dataset
- Identificar os nomes das colunas disponíveis
- Realizar uma primeira checagem da integridade básica da estrutura de dados

In [4]:
# PARA MELHOR VISUALIZAÇÃO
print("Número de linhas:", df_inicial.shape[0])
print("Número de colunas:", df_inicial.shape[1])
print("\nColunas disponíveis:")
for i, col in enumerate(df_inicial.columns, 1):
    print(f"{i}. {col}")

Número de linhas: 557586
Número de colunas: 19

Colunas disponíveis:
1. date_message
2. id_member_anonymous
3. id_group_anonymous
4. media
5. media_type
6. media_url
7. has_media
8. has_media_url
9. trava_zap
10. text_content_anonymous
11. dataset_info_id
12. date_system
13. score_sentiment
14. score_misinformation
15. id_message
16. message_type
17. messenger
18. media_name
19. media_md5


O método `display()` do IPython renderiza objetos como DataFrames de forma mais rica que print(), mantendo a formatação tabular. Combinado com `.style`, permite personalizar a aparência: `.set_caption()` adiciona títulos descritivos, enquanto `.set_properties()` aplica estilos CSS como cores de fundo suaves (#f8f9fa), bordas sutis, truncamento de texto longo (text-overflow: ellipsis) e limitação de largura (max-width), melhorando a legibilidade e organização dos dados exibidos.

Essa abordagem foi utilizada, pois é importante ter em mente as primeiras e últimas amostras do dataset.

In [5]:
# VISUALIZANDO AMOSTRAS INICIAIS E FINAIS DO DATASET
display(df_inicial.head().style.set_caption("Primeiros Registros").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '300px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))

display(df_inicial.tail().style.set_caption("Últimos Registros").set_properties(**{
    'background-color': '#f8f9fa',
    'border': '1px solid #dee2e6',
    'color': '#212529',
    'max-width': '300px',
    'overflow': 'hidden',
    'text-overflow': 'ellipsis',
    'white-space': 'nowrap'
}))

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,Então é Fato Renato o áudio que eu ouvi no whatsapp isso ocorreu em Niterói principalmente no bairro Fonseca ?,5,2022-10-05 06:25:28.863641,0.000000,nan,16385,Texto,telegram,nan,nan
1,2022-10-05 06:25:08,nan,12283e08a2eb5789201e105b34489ee7,nan,nan,nan,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás, infelizmente não consigo enviar para cá, mas é facilmente verificável no YouTube do presidente",5,2022-10-05 06:25:28.926311,0.064400,nan,16386,Texto,telegram,nan,nan
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,nan,nan,nan,False,False,False,"É isso, nossa parte já foi quase toda feita. No segundo turno completamos nossa parte desse teatro. Essa é uma guerra de 4* geração na dimensão humana e uma guerra espiritual do bem contra o mal na dimensão do Universo. Pensamento positivo é fundamental, pensem sempre em algo bom. Deus continua nos abençoando, nosso livre arbítrio completa o curso.",5,2022-10-05 06:26:29.361949,-0.355100,0.157242,16366,Texto,telegram,nan,nan
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,nan,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.000000,nan,19281,Imagem,telegram,nan,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,nan,True,False,False,nan,5,2022-10-05 06:28:29.316325,nan,nan,507185,Imagem,telegram,nan,5ad5c8bbe9da93a37fecf3e5aa5b0637


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
557581,2022-11-11 12:06:15,333e9869f23dbd4682d1be382d9c1e59,e56ec342fc599ebb4ed89655eb6f03aa,25e43b6a58b848c43ad5b5f9e979822a.jpg,url,https://terrabrasilnoticias.com/2022/11/bndes-tem-lucro-de-r-96-bilhoes-no-terceiro-trimestre/,True,True,False,"BNDES tem lucro de R$ 9,6 bilhões no terceiro trimestre ☛ https://terrabrasilnoticias.com/2022/11/bndes-tem-lucro-de-r-96-bilhoes-no-terceiro-trimestre/",5,2022-11-16 14:49:39.146502,0.102700,nan,575796,Url,telegram,nan,25e43b6a58b848c43ad5b5f9e979822a
557582,2022-11-11 12:09:08,nan,5b10d7739171149be6d9961e3350c071,657949d03e4088f6b332e2686ccd3221.jpg,url,https://youtu.be/8g1Vz9_0xVk,True,True,False,https://youtu.be/8g1Vz9_0xVk,5,2022-11-16 14:49:39.847434,0.000000,nan,1286443,Url,telegram,nan,657949d03e4088f6b332e2686ccd3221
557583,2022-11-11 12:09:47,nan,1590a03f43b5ba4b6147a1c5e1dd357b,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:39.922279,0.000000,nan,13294,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e
557584,2022-11-11 12:09:46,nan,5b10d7739171149be6d9961e3350c071,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:39.992932,0.000000,nan,1286444,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e
557585,2022-11-11 12:09:48,nan,b11f2df64ac19aad47a50accf32052d6,a21848a61045380a6483866daed0ca0e.jpg,image/jpg,https://t.me/vemprasruas,True,True,False,"Empresários, demitam os petistas primeiro. https://t.me/vemprasruas",5,2022-11-16 14:49:40.064006,0.000000,nan,192127,Imagem,telegram,nan,a21848a61045380a6483866daed0ca0e


---

### **(b)** Remova os trava-zaps.

Este item será resolvido através da manipulação da coluna `trava_zap`, previamente identificada na análise inicial. A estratégia adotada considera o tamanho do dataset (557.586 registros) e segue um fluxo estruturado:

1. **Análise Preliminar**
    - Verificação do tipo de dado (dtype) e valores únicos na coluna `trava_zap`
    - Confirmação da proporção True/False/NaN (se aplicável)

In [6]:
# ANÁLISE PRELIMINAR
# Verificação do tipo e valores únicos
print(f"Tipo de dado: {df_inicial['trava_zap'].dtype}")
trava_dist = df_inicial['trava_zap'].value_counts(dropna=False)
display(trava_dist.to_frame().style.set_caption("Contagem de valores distintos em trava_zap"))

# Verificação de linhas que devem ser removidas
print(f"Registros com trava_zap que precisam ser removidos: {trava_dist.get(True, 0):,}")

Tipo de dado: bool


,count
trava_zap,
False,557570
True,16


Registros com trava_zap que precisam ser removidos: 16


2. **Processamento por Chunks**
    - Divisão do dataset em blocos de 1.000 linhas para:
        - Otimização de memória
        - Facilidade de debug
        - Monitoramento do progresso

In [7]:
import numpy as np

In [8]:
# Parâmetros do chunk -- 1000 linhas 
CHUNK_SIZE = 1000
total_chunks = (len(df_inicial) // CHUNK_SIZE) + 1
df_sem_trava_zap = pd.DataFrame() # Criação de DataFrame Seguro para controle de variáveis.

print(f"\033[1mProcessando {total_chunks} chunks de {CHUNK_SIZE} registros cada:\033[0m")

for i, chunk in enumerate(np.array_split(df_inicial, total_chunks)):
    # Filtro principal -- linhas que possuem trava zap, isto é, trava_zap = True
    chunk_filtrado = chunk[chunk['trava_zap'] != True]
    
    # Concatenção segura -- recebendo apenas os blocos onde trava_zap = False
    df_sem_trava_zap = pd.concat([df_sem_trava_zap, chunk_filtrado], ignore_index=True)
    
    # Log de progresso para controle da análise
    if (i+1) % 10 == 0:
        print(f"Processado chunk {i+1}/{total_chunks} | Registros retidos: {len(df_sem_trava_zap):,}")

Processando 558 chunks de 1000 registros cada:


c:\Users\esthe\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


Processado chunk 10/558 | Registros retidos: 10,000
Processado chunk 20/558 | Registros retidos: 20,000
Processado chunk 30/558 | Registros retidos: 29,999
Processado chunk 40/558 | Registros retidos: 39,999
Processado chunk 50/558 | Registros retidos: 49,999
Processado chunk 60/558 | Registros retidos: 59,999
Processado chunk 70/558 | Registros retidos: 69,999
Processado chunk 80/558 | Registros retidos: 79,999
Processado chunk 90/558 | Registros retidos: 89,998
Processado chunk 100/558 | Registros retidos: 99,998
Processado chunk 110/558 | Registros retidos: 109,998
Processado chunk 120/558 | Registros retidos: 119,998
Processado chunk 130/558 | Registros retidos: 129,998
Processado chunk 140/558 | Registros retidos: 139,998
Processado chunk 150/558 | Registros retidos: 149,992
Processado chunk 160/558 | Registros retidos: 159,982
Processado chunk 170/558 | Registros retidos: 169,972
Processado chunk 180/558 | Registros retidos: 179,962
Processado chunk 190/558 | Registros retidos: 1

4. **Saída Controlada**
    - Relatório pós-processamento mostrará:
        - Número total de linhas removidas
        - Porcentagem impactada
        - Amostra das primeiras 5 linhas removidas (se relevante)

In [9]:
# Relatório final
print("\033[1mRelatório de Remoção:\033[0m")
print(f"• Registros originais: {len(df_inicial):,}")
print(f"• Registros removidos: {trava_dist.get(True, 0):,}")
print(f"• Registros restantes: {len(df_sem_trava_zap):,}") 

# Visualização dos dados que foram removidos
if trava_dist.get(True, 0) > 0:
    display(
        df_inicial[df_inicial['trava_zap'] == True].head(16)  
    )

Relatório de Remoção:
• Registros originais: 557,586
• Registros removidos: 16
• Registros restantes: 557,570


,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
21944,2022-10-07 07:46:52,NaN,c712c1b704c22bd0cef50bc06125cdbd,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-07 07:47:00.355052,0.0000,0.067344,53260,Texto,telegram,NaN,NaN
89109,2022-10-16 00:45:02,8a30ac374bc4b5930eaf0667a178546a,e56ec342fc599ebb4ed89655eb6f03aa,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-16 00:45:04.064662,0.0000,0.056698,521324,Texto,telegram,NaN,NaN
294541,2022-10-04 14:22:47,39ee10516124280a22f1798f2a41f9a7,959f13e0079883060632c74ffc81c547,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-04 14:22:48.808572,0.9734,0.010433,27241,Texto,telegram,NaN,NaN
324567,2022-10-25 14:55:55,e003fbb6ffedb1838e42360d41cab314,5b10d7739171149be6d9961e3350c071,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-25 14:56:11.604972,0.9460,0.403945,1182938,Texto,telegram,NaN,NaN
389164,2022-10-30 20:19:52,NaN,c8f2de56550ed0bf85249608b7ead93d,NaN,NaN,NaN,False,False,True,NaN,5,2022-10-30 20:19:54.183578,0.0000,NaN,28330,Texto,telegram,NaN,NaN
423083,2022-11-03 00:40:23,NaN,4d3712f5a117e36180d4b4cbd07c540e,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-03 00:40:48.993091,0.0000,0.067026,171104,Texto,telegram,NaN,NaN
466735,2022-11-07 20:47:35,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-07 20:48:16.653321,0.0000,0.036173,88890,Texto,telegram,NaN,NaN
467297,2022-11-08 10:07:14,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-08 10:07:25.260375,0.0000,0.072637,88977,Texto,telegram,NaN,NaN
471273,2022-11-08 20:37:52,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-08 20:38:02.773746,0.0000,0.031973,89039,Texto,telegram,NaN,NaN
478270,2022-11-09 19:50:12,4a498818da925377eff2606a260cfa45,f61777908059b318385882ff47b15c33,NaN,NaN,NaN,False,False,True,NaN,5,2022-11-09 19:50:36.039894,0.0000,0.029651,89212,Texto,telegram,NaN,NaN


---

### **(c)** Exportar os dados para um arquivo Parquet. 

Existe um método do pandas chamado `to_parquet`, sendo a forma mais eficiente de exportar DataFrames para o formato Parquet. Sua sintaxe básica inclui dois parâmetros essenciais:

```bash
df.to_parquet(
    path='dados.parquet',  # Caminho do arquivo de saída
    engine='auto'         # Motor de processamento (pyarrow/fastparquet)
)
```

**Observação**: O `engine='auto'` deixa o Pandas escolher automaticamente o motor disponível (geralmente PyArrow ou Fastparquet), mas eu costumo usar o pyarrow explicitamente porque ele é mais estável e tem suporte a mais funcionalidades. Além disso, ele permite leitura e escrita super rápidas, e mantém os tipos de dados de forma mais fiel.

In [ ]:
df_sem_trava_zap.to_parquet('fakeTelegram.BR_2022_filtrado.parquet', engine='pyarrow')

---

### **(d)** Exportar os dados para o DuckDB. 

Nesta etapa, exportei o DataFrame anterior para um banco de dados DuckDB, que é um sistema de gerenciamento de banco de dados analítico leve e embutido. Com isso, realizei alguns passos importantes.

Primeiramente, fiz a instalação e a importação do DuckDB:

In [10]:
# !pip install duckdb -- Apenas se ainda não estiver instalado
import duckdb

Criação (ou conexão) com o banco DuckDB:

In [11]:
conn = duckdb.connect("fakeTelegram.BR_2022_dados.duckdb") 

Exportação do `df_sem_trava_zap` para uma tabela do banco:

In [12]:
conn.register("df_sem_trava_zap", df_sem_trava_zap)  # Registra o DataFrame como uma "tabela temporária"
conn.execute("CREATE TABLE IF NOT EXISTS tabela_dados AS SELECT * FROM df_sem_trava_zap")  # Exporta os dados

Verificação dos dados no banco:

In [13]:
conn.execute("SELECT * FROM tabela_dados LIMIT 5").fetchdf()

,date_message,id_member_anonymous,id_group_anonymous,media,media_type,media_url,has_media,has_media_url,trava_zap,text_content_anonymous,dataset_info_id,date_system,score_sentiment,score_misinformation,id_message,message_type,messenger,media_name,media_md5
0,2022-10-05 06:25:04,1078cc958f0febe28f4d03207660715f,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,False,Então é Fato Renato o áudio que eu ouvi no wha...,5,2022-10-05 06:25:28.863641,0.0000,NaN,16385,Texto,telegram,None,None
1,2022-10-05 06:25:08,None,12283e08a2eb5789201e105b34489ee7,None,None,None,False,False,False,"Saiu no YouTube do presidente a 8 horas atrás,...",5,2022-10-05 06:25:28.926311,0.0644,NaN,16386,Texto,telegram,None,None
2,2022-10-05 06:26:28,92a2d8fd7144074f659d1d29dc3751da,9f2d7394334eb224c061c9740b5748fc,None,None,None,False,False,False,"É isso, nossa parte já foi quase toda feita. N...",5,2022-10-05 06:26:29.361949,-0.3551,0.157242,16366,Texto,telegram,None,None
3,2022-10-05 06:27:28,d60aa38f62b4977426b70944af4aff72,c8f2de56550ed0bf85249608b7ead93d,94dca4cda503100ebfda7ce2bcc060eb.jpg,image/jpg,None,True,False,False,GENTE ACHEI ELES EM UMA SEITA MAÇONÁRICA,5,2022-10-05 06:27:29.935624,0.0000,NaN,19281,Imagem,telegram,None,94dca4cda503100ebfda7ce2bcc060eb
4,2022-10-05 06:27:44,cd6979b0b5265f08468fa1689b6300ce,e56ec342fc599ebb4ed89655eb6f03aa,5ad5c8bbe9da93a37fecf3e5aa5b0637.jpg,image/jpg,None,True,False,False,None,5,2022-10-05 06:28:29.316325,NaN,NaN,507185,Imagem,telegram,None,5ad5c8bbe9da93a37fecf3e5aa5b0637


Como os próximos itens são de recuperação de dados utilizando o DuckDB, não vamos encerrar a conexão neste item, mas sim no final do processo.

---

### **(e)** Utlizando o DuckDB recupere:

#### **1**. A quantidade de mensagens

Para resolver este item, podemos utilizar de consultas SQL. A primeira consulta realizada foi para contar o total de mensagens armazenadas na tabela `tabela_dados` do banco DuckDB. Cada linha da tabela representa uma mensagem única, enviada em grupos do Telegram relacionados ao contexto do dataset analisado.
> A tabela `tabela_dados` contém **557.570 mensagens** e **19 atributos**, entre eles: `date_message`, `id_group_anonymous`, `has_media`, `trava_zap`, `text_content_anonymous`, entre outros que serão úteis nas análises a seguir.

In [14]:
qtd_mensagem = conn.execute("SELECT COUNT(*) AS total_mensagens FROM tabela_dados").fetchdf()
display(qtd_mensagem)

,total_mensagens
0,557570


Esta consulta realiza uma contagem multidimensional das mensagens, fornecendo diferentes perspectivas sobre a composição do dataset:

In [15]:
# text_content_anonymous leva em consideração apenas mensagem de texto
qtd_mensagem_adicional = conn.execute("""
    SELECT 
    COUNT(id_message) AS mensagens_com_id_valido,
    COUNT(DISTINCT id_message) AS mensagens_unicas,
    SUM(CASE WHEN text_content_anonymous IS NOT NULL THEN 1 ELSE 0 END) AS mensagens_textuais,
    SUM(CASE WHEN has_media = TRUE THEN 1 ELSE 0 END) AS mensagens_com_midia
    FROM tabela_dados
""").fetchdf()

display(qtd_mensagem_adicional)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,mensagens_com_id_valido,mensagens_unicas,mensagens_textuais,mensagens_com_midia
0,557570,364000,444201.0,332604.0


---

#### **2**. A quantidade de usuários

Para recuperar a quantidade de usuários únicos no dataset, utilizamos a coluna `id_member_anonymous`, que contém identificadores anônimos dos membros que enviaram mensagens nos grupos do Telegram. A consulta SQL no DuckDB foi a seguinte:

In [16]:
# Consulta para contar usuários únicos
qtd_usuarios = conn.execute("""
    SELECT 
    COUNT(DISTINCT id_member_anonymous) AS total_usuarios_unicos,
    COUNT(*) AS total_registros,
    ROUND(COUNT(DISTINCT id_member_anonymous) * 100.0 / COUNT(*), 2) AS percentual_identificados
    FROM tabela_dados
""").fetchdf()

display(qtd_usuarios)

,total_usuarios_unicos,total_registros,percentual_identificados
0,14809,557570,2.66


**Observações importantes**

- Algumas mensagens possuem `id_member_anonymous` como `NULL`, indicando que o remetente é anônimo ou não identificado.

- O número de usuários únicos é menor que o total de mensagens, pois um mesmo usuário pode ter enviado múltiplas mensagens.

In [17]:
qtd_sem_id = conn.execute("""
    SELECT 
    COUNT(CASE WHEN id_member_anonymous IS NULL THEN 1 END) AS mensagens_anonimas,
    COUNT(CASE WHEN id_member_anonymous IS NOT NULL THEN 1 END) AS mensagens_identificadas,
    COUNT(DISTINCT id_member_anonymous) AS usuarios_unicos_identificados
FROM tabela_dados
""").fetchdf()

display(qtd_sem_id)

,mensagens_anonimas,mensagens_identificadas,usuarios_unicos_identificados
0,323337,234233,14809


**Relação quantitativa:**

- Total de mensagens não nulas: 444.201

- Usuários únicos: 14.809

- Mensagens sem identificação: 323.337

---

#### **3**. A quantidade de grupos

Para identificar a quantidade de grupos únicos no dataset, utilizamos a coluna `id_group_anonymous` que contém identificadores únicos dos grupos do Telegram.

In [18]:
qtd_grupos = conn.execute("SELECT COUNT(DISTINCT id_group_anonymous) AS total_grupos_unicos FROM tabela_dados").fetchdf()
display(qtd_grupos)

,total_grupos_unicos
0,178


Sobre a coluna `id_group_anonymous`:

- **Valores**: São hashes MD5 únicos para cada grupo.
- **NULL**: Indica grupos não identificados (raro em seu dataset).

Relação com outras métricas:

- Cada grupo contém múltiplos `id_member_anonymous` (membros).
- Está relacionado com `message_type` (tipo de conteúdo predominante no grupo).

Dados técnicos:

- **Cardinalidade esperada**: Muito menor que o total de mensagens.
- **Tipo de dado**: TEXT (32 caracteres hexadecimais).

In [19]:
distribuicao_grupos = conn.execute("""
    SELECT 
    COUNT(DISTINCT id_group_anonymous) AS grupos_unicos,
    COUNT(*) AS total_mensagens,
    ROUND(COUNT(*) * 1.0 / COUNT(DISTINCT id_group_anonymous), 2) AS media_mensagens_por_grupo
    FROM tabela_dados                                                        
""").fetchdf()

display(distribuicao_grupos)

,grupos_unicos,total_mensagens,media_mensagens_por_grupo
0,178,557570,3132.42


---

#### **4**. Quantidade de mensagens que possuem apenas texto

Esta métrica identifica mensagens **exclusivamente textuais**, ou seja, que não contêm nenhum tipo de mídia anexada. Portanto, utilizei dados de duas colunas: `text_content_anonymous` e `has_media = FALSE`.

```sql
SELECT
    COUNT(*) AS mensagens_apenas_texto
FROM tabela_dados
WHERE 
    text_content_anonymous IS NOT NULL  -- Possui conteúdo textual
    AND has_media = FALSE               -- Não possui mídia anexada

In [20]:
qtd_mensagem_texto = conn.execute("""
    SELECT
    COUNT(*) AS total_mensagens,
    SUM(CASE WHEN text_content_anonymous IS NOT NULL AND has_media = FALSE THEN 1 ELSE 0 END) AS mensagens_apenas_texto,
    ROUND(SUM(CASE WHEN text_content_anonymous IS NOT NULL AND has_media = FALSE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS percentual_apenas_texto
    FROM tabela_dados                             
""").fetchdf()

display(qtd_mensagem_texto)

,total_mensagens,mensagens_apenas_texto,percentual_apenas_texto
0,557570,224966.0,40.35


#### **5**. Quantidade de mensagens contendo mídias

Utilizei a mesma lógica do item anterior para criar a consulta SQL, porém, agora estou pegando os dados com `has_media = TRUE` e `text_content_anonymous = NULL`, ou seja, não há mensagens de texto.

In [21]:
qtd_mensagem_midias = conn.execute("""
    SELECT
    COUNT(*) AS total_mensagens,
    SUM(CASE WHEN text_content_anonymous IS NULL AND has_media = TRUE THEN 1 ELSE 0 END) AS mensagens_apenas_midia,
    ROUND(SUM(CASE WHEN text_content_anonymous IS NULL AND has_media = TRUE THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS percentual_apenas_midia
    FROM tabela_dados                             
""").fetchdf()

display(qtd_mensagem_midias)

,total_mensagens,mensagens_apenas_midia,percentual_apenas_midia
0,557570,113369.0,20.33


---

#### **6**. Quantidade de mensagens por tipo de mídia (jpg, mp4 etc)

Para calcular essa métrica, segui os seguintes passos:

1. Filtrar mensagens com `has_media = TRUE`.
2. Ver quais valores únicos existem na coluna `media_type` (isso indica os tipos: 'jpg', 'mp4', 'pdf', etc.).
3. Agrupar por `media_type` e contar a quantidade.

In [22]:
qtd_tipo_midia = conn.execute("""
    SELECT 
    media_type,
    COUNT(*) AS qtd_mensagens
    FROM tabela_dados
    WHERE has_media = TRUE
    GROUP BY media_type
    ORDER BY qtd_mensagens DESC      
""").fetchdf()

display(qtd_tipo_midia)

,media_type,qtd_mensagens
0,image/jpg,200440
1,url,100856
2,video/mp4,18497
3,application/vnd.android.package-archive,7159
4,application/pdf,2850
...,...,...
62,application/x-msi,1
63,image/m4a,1
64,image/wma,1
65,image/af75130da3b6516eb3c1eb9ab62b5219,1


---

#### **12**. As 30 URLs que mais se repetem (mais compartilhadas)

É possível realizar esse cálculo pegando os dados da coluna `media_url`, pois essa feature provavelmente contém os links das mídias compartilhadas.

In [23]:
urls_mais_compartilhadas = conn.execute("""
    SELECT 
    media_url,
    COUNT(*) AS quantidade
    FROM tabela_dados
    WHERE media_url IS NOT NULL
    GROUP BY media_url
    ORDER BY quantidade DESC
    LIMIT 30                        
""").fetchdf()

display(urls_mais_compartilhadas)

,media_url,quantidade
0,t.me/alexeconomia,4159
1,https://youtube.com/c/especulandoosfatosoficial,1607
2,t.me/fimtaproximo,1581
3,t.me/+EWlGMatRZGg3OTlh,1419
4,https://youtu.be/qbTzhB0akt8,1160
5,https://youtu.be/zDuOoyhyN-4,1022
6,t.me/+ewZIPdZ42vEyNzJh,772
7,https://t.me/canalselvabrasiloficial,709
8,https://youtu.be/4DHk9KZ01HM,660
9,T.me/Arthur_Senna_Trader2,640


---

#### **13**. Os 30 domínios que mais se repetem (mais compartilhados)

Para identificar os sites mais utilizados como fonte de mídia nas mensagens, foi realizada uma extração dos domínios contidos nas URLs presentes na coluna `media_url`.

A base de dados não possui uma coluna explícita de localização ou domínio, mas como a coluna `media_url` armazena os links das mídias compartilhadas, foi possível utilizar expressões regulares (**regex**) para extrair o domínio principal (ex: youtube.com, t.me, wa.me) de cada URL.

In [24]:
dominios_mais_compartilhados = conn.execute("""
    SELECT
    CASE
        WHEN media_url LIKE 'http://%' THEN SUBSTRING(media_url, 8, POSITION('/' IN SUBSTRING(media_url, 8)) - 1)
        WHEN media_url LIKE 'https://%' THEN SUBSTRING(media_url, 9, POSITION('/' IN SUBSTRING(media_url, 9)) - 1)
        ELSE media_url
    END AS dominio,
    COUNT(*) AS quantidade
FROM tabela_dados
WHERE media_url IS NOT NULL
GROUP BY dominio
ORDER BY quantidade DESC
LIMIT 30;                                   
""").fetchdf()

display(dominios_mais_compartilhados)

,dominio,quantidade
0,youtu.be,52731
1,t.me,10712
2,www.youtube.com,8315
3,terrabrasilnoticias.com,8282
4,youtube.com,5967
5,www.instagram.com,5784
6,www.jornaldacidadeonline.com.br,4829
7,twitter.com,4390
8,t.me/alexeconomia,4159
9,gazetabrasil.com.br,1779


Também busquei realizar um tratamento adicional para unificar domínios que se referiam à mesma plataforma, por exemplo, `youtu.be` e `youtube.com`.

In [25]:
dominios_mais_compartilhados_update = conn.execute("""
   WITH dominios_unificados AS (
    SELECT
        CASE
            -- Unifica variações do YouTube
            WHEN LOWER(media_url) LIKE '%youtube.com%' OR LOWER(media_url) LIKE '%youtu.be%' 
            THEN 'youtube.com'
            
            -- Unifica variações do Telegram
            WHEN LOWER(media_url) LIKE '%t.me%' OR LOWER(media_url) LIKE '%T.me%'
            THEN 't.me'
            
            -- Unifica variações do Instagram
            WHEN LOWER(media_url) LIKE '%instagram.com%' 
            THEN 'instagram.com'
            
            -- Unifica variações do Twitter
            WHEN LOWER(media_url) LIKE '%twitter.com%' OR LOWER(media_url) LIKE '%x.com%' 
            THEN 'twitter.com'
            
            -- Remove 'www.' e outros subdomínios para casos gerais
            WHEN media_url LIKE 'http://%' OR media_url LIKE 'https://%' 
            THEN REGEXP_REPLACE(
                REGEXP_REPLACE(media_url, '^https?://(www\.)?([^/]+).*', '\2'),
                '^([^.]+\.)?([a-z-]+\.[a-z]{2,6})$', '\2'
            )
            
            ELSE media_url
        END AS dominio_unificado,
        media_url
    FROM tabela_dados
    WHERE media_url IS NOT NULL AND media_url != ''
    )
    SELECT
        dominio_unificado AS dominio,
        COUNT(*) AS quantidade,
        ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM dominios_unificados), 2) AS percentagem
    FROM dominios_unificados
    GROUP BY dominio_unificado
    ORDER BY quantidade DESC
    LIMIT 30;                            
""").fetchdf()

display(dominios_mais_compartilhados_update)

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\Users\esthe\AppData\Local\Temp\ipykernel_16288\3980102472.py:1: SyntaxWarning: invalid escape sequence '\.'
  dominios_mais_compartilhados_update = conn.execute("""


,dominio,quantidade,percentagem
0,youtube.com,68312,43.39
1,,55182,35.05
2,t.me,21377,13.58
3,instagram.com,6671,4.24
4,twitter.com,4863,3.09
5,www.ceifadores.com.br,77,0.05
6,www.marketingdigitalparavencer.com.br,73,0.05
7,www.fiscaisdobolsonaro.com.br,48,0.03
8,brazilwasstolen.com/pt/,42,0.03
9,www.starday.com.br/crp,39,0.02


---

#### **14**. Os 30 usuários mais ativos

Essa consulta foi construída para descobrir quem são os 30 usuários mais ativos com base no volume de mensagens que enviaram. Eu dividi a análise em duas partes principais usando uma Common Table Expression (CTE), que é esse bloco `WITH atividade_usuarios AS (...)`. Além disso, a coluna `id_member_anonymous` é fundamental nessa consulta — e em qualquer análise de comportamento de usuários — porque ela representa o identificador único de cada usuário, mesmo que de forma anônima.

Essa consulta me dá um retrato bem completo dos usuários mais ativos da base, não só em volume, mas em variedade de atividade (dias, grupos, tipos de mídia). E ainda consigo comparar o peso de cada um no total com esse percentual.

In [26]:
users_mais_ativos = conn.execute("""
    WITH atividade_usuarios AS (
    SELECT
        id_member_anonymous AS usuario,
        COUNT(*) AS total_mensagens,
        COUNT(DISTINCT id_group_anonymous) AS grupos_ativos,
        COUNT(DISTINCT CAST(date_message AS DATE)) AS dias_ativos,
        MIN(date_message) AS primeira_atividade,
        MAX(date_message) AS ultima_atividade,
        COUNT(DISTINCT message_type) AS tipos_conteudo,
        SUM(CASE WHEN has_media = TRUE THEN 1 ELSE 0 END) AS midias_enviadas
    FROM tabela_dados
    WHERE id_member_anonymous IS NOT NULL
    GROUP BY id_member_anonymous
    )
    SELECT
        usuario,
        total_mensagens,
        grupos_ativos,
        dias_ativos,
        primeira_atividade,
        ultima_atividade,
        tipos_conteudo,
        midias_enviadas,
        ROUND(total_mensagens * 100.0 / (SELECT SUM(total_mensagens) FROM atividade_usuarios), 2) AS percentual_total,
        RANK() OVER (ORDER BY total_mensagens DESC) AS ranking
    FROM atividade_usuarios
    ORDER BY total_mensagens DESC
    LIMIT 30;
""").fetchdf()

display(users_mais_ativos)

,usuario,total_mensagens,grupos_ativos,dias_ativos,primeira_atividade,ultima_atividade,tipos_conteudo,midias_enviadas,percentual_total,ranking
0,abe534d581ec6d552243d6955d3c3cd8,12289,8,44,2022-09-29 00:00:15,2022-11-11 11:16:39,1,0.0,5.25,1
1,1665e22b0f564cd46d343f7677014821,5452,14,44,2022-09-29 00:34:32,2022-11-11 11:58:44,2,2972.0,2.33,2
2,1ac091b8ed5c4e42383f1b4ff4cc9b2d,5060,2,36,2022-09-29 00:49:50,2022-11-03 19:43:48,5,4632.0,2.16,3
3,c743967449a387ad2c1c7e03b2c45b36,3019,1,38,2022-10-05 15:00:20,2022-11-11 11:00:40,2,152.0,1.29,4
4,e7998863ac2a40086657fab4a6b463c9,1928,2,39,2022-10-04 22:27:13,2022-11-11 12:01:04,5,1498.0,0.82,5
5,e8fd8fee8c39342d37993775da7756d5,1706,1,43,2022-09-29 11:11:51,2022-11-10 22:53:26,3,1007.0,0.73,6
6,2f4be6244ede15b46e8329a2c975be30,1620,2,44,2022-09-29 00:19:30,2022-11-11 03:52:55,3,1489.0,0.69,7
7,d49f81df0c75d1d72bee6c5b2d707da0,1571,2,44,2022-09-29 01:02:26,2022-11-11 10:01:23,1,0.0,0.67,8
8,4f7d493f0f6222d56e5b19a4f7c336cc,1447,8,41,2022-09-29 02:52:53,2022-11-11 01:32:21,4,1359.0,0.62,9
9,773b9bd5b02a2e96f9d732c29bfcb663,1411,1,43,2022-09-29 09:26:19,2022-11-11 09:43:53,3,1167.0,0.60,10


---

#### **15**. Os 30 usuários que mais compartilharam texto

Essa consulta conta quantas mensagens de texto puro cada usuário enviou, ignorando as que têm mídia. Usei `has_media = FALSE` pra garantir que são só textos, e também filtrei pra que o campo de texto não esteja vazio. Depois agrupei tudo por usuário e ordenei pra pegar os 30 que mais mandaram texto.

In [27]:
users_mais_texto = conn.execute("""
    SELECT
    id_member_anonymous AS usuario,
    COUNT(*) AS total_textos
    FROM tabela_dados
    WHERE
        id_member_anonymous IS NOT NULL
        AND has_media = FALSE
        AND text_content_anonymous IS NOT NULL
        AND TRIM(text_content_anonymous) <> ''
    GROUP BY id_member_anonymous
    ORDER BY total_textos DESC
    LIMIT 30;            
""").fetchdf()

display(users_mais_texto)

,usuario,total_textos
0,abe534d581ec6d552243d6955d3c3cd8,12289
1,c743967449a387ad2c1c7e03b2c45b36,2867
2,1665e22b0f564cd46d343f7677014821,2480
3,d49f81df0c75d1d72bee6c5b2d707da0,1571
4,f233cf8b1d4ede06f32199a7e0081bf5,1022
5,e8fd8fee8c39342d37993775da7756d5,699
6,a378e9743fa3ca297df321cfa0e7cf6a,598
7,7696d5103cdb8ac352d748a1db1126b0,560
8,26b496125c8b6773bce453e14f172430,502
9,7d41f1a2df245b0cc870f546c2aead0b,483


---

#### **16**. Os 30 usuários que mais compartilharam mídias

Aqui eu filtrei só as mensagens que têm mídia (`has_media = TRUE`) e agrupei por usuário. Depois contei quantas mídias cada um compartilhou, ordenei do maior pro menor e peguei os 30 mais ativos nesse tipo de conteúdo.

In [28]:
users_mais_midias = conn.execute("""
    SELECT
    id_member_anonymous AS usuario,
    COUNT(*) AS total_midias
    FROM tabela_dados
    WHERE
        id_member_anonymous IS NOT NULL
        AND has_media = TRUE
    GROUP BY id_member_anonymous
    ORDER BY total_midias DESC
    LIMIT 30;            
""").fetchdf()

display(users_mais_midias)

,usuario,total_midias
0,1ac091b8ed5c4e42383f1b4ff4cc9b2d,4632
1,1665e22b0f564cd46d343f7677014821,2972
2,e7998863ac2a40086657fab4a6b463c9,1498
3,2f4be6244ede15b46e8329a2c975be30,1489
4,4f7d493f0f6222d56e5b19a4f7c336cc,1359
5,c052c859b42c5a1923c22f5a201de746,1289
6,773b9bd5b02a2e96f9d732c29bfcb663,1167
7,56b8359fd127312651b80b8ed8030085,1079
8,e8fd8fee8c39342d37993775da7756d5,1007
9,3e49fd40fd973ee1b8f1a6d58feb4a54,965


---

#### **17**. As 30 mensagens mais compartilhadas

Nessa consulta, agrupei as mensagens textuais que aparecem repetidas na base e contei quantas vezes cada uma foi compartilhada. Filtrei mensagens vazias ou nulas e ordenei pra ver quais conteúdos circularam mais vezes. O resultado mostra as 30 mensagens mais replicadas pelos usuários.

In [29]:
mensagens_mais_compartilhadas = conn.execute("""
    SELECT
    text_content_anonymous AS mensagem,
    COUNT(*) AS total_compartilhamentos
    FROM tabela_dados
    WHERE
        text_content_anonymous IS NOT NULL
        AND TRIM(text_content_anonymous) <> ''
    GROUP BY text_content_anonymous
    ORDER BY total_compartilhamentos DESC
    LIMIT 30;         
""").fetchdf()

display(mensagens_mais_compartilhadas)

,mensagem,total_compartilhamentos
0,This community was blocked in Brazil following...,17422
1,Rough_sex🙈,1134
2,Anal sex🙈,1118
3,سکس مردان ازبک با زن انگلیسی با این vpn از سای...,1019
4,https://youtu.be/qbTzhB0akt8,758
5,https://youtu.be/zDuOoyhyN-4,712
6,فیلم سوپر با زیرنویس فارسی ببین😍😍\nبا این فیلت...,632
7,Rough😱,570
8,Foto de Nélia Barros,548
9,We had no choice but to remain in the shadows....,480


---

#### **18**. As 30 mensagens mais compartilhadas em grupos diferentes

Essa consulta identifica as mensagens textuais que apareceram em mais grupos diferentes. Isso mostra quais conteúdos se espalharam mais pela rede, não só por número de envios, mas em quantidade de grupos distintos. Também incluí a contagem total de vezes que cada mensagem apareceu, só como referência.

In [30]:
mensagems_grupos = conn.execute("""
    SELECT
    text_content_anonymous AS mensagem,
    COUNT(DISTINCT id_group_anonymous) AS total_grupos_diferentes,
    COUNT(*) AS total_ocorrencias
    FROM tabela_dados
    WHERE
        text_content_anonymous IS NOT NULL
        AND TRIM(text_content_anonymous) <> ''
    GROUP BY text_content_anonymous
    ORDER BY total_grupos_diferentes DESC
    LIMIT 30;
""").fetchdf()

display(mensagems_grupos)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,mensagem,total_grupos_diferentes,total_ocorrencias
0,This community was blocked in Brazil following...,59,17422
1,BOA NOITE A TODOS. O QUE EU VOU DIZER É SÉRIO ...,36,127
2,Hoje às 18 horas no canal Inteligência Ltda do...,35,37
3,[USER],32,170
4,https://youtu.be/zDuOoyhyN-4,31,712
5,https://youtu.be/qbTzhB0akt8,30,758
6,OS PTISTAS NÃO QUER QUE AGENTE MOSTRE QUE VOTO...,29,33
7,*ATENÇÃO* \n🚨🚨🚨🚨🚨🚨🚨🚨🚨\n\n*CHAMADA URGENTE* Pat...,29,33
8,"DE NADA IRÁ ADIANTAR FISCALIZAR AS URNAS, SE N...",29,30
9,Ajudem a subir a #JanonesAmigoDePedofilo\n\nPo...,29,43


---

#### **19**. Mensagens idênticas compartilhadas pelo mesmo usuário (e suas quantidades) 

Essa consulta mostra os casos em que um mesmo usuário compartilhou exatamente a mesma mensagem mais de uma vez. Agrupei por usuário, texto e data, contei as repetições e usei `HAVING COUNT(*) > 1` pra filtrar só as duplicações. O resultado revela comportamentos de possível spam ou insistência no mesmo conteúdo

In [32]:
qtd_mensagens_identicas = conn.execute("""
SELECT
    id_member_anonymous AS usuario,
    CAST(date_message AS DATE) as data,
    text_content_anonymous AS mensagem,
    COUNT(*) AS quantidade_envios
FROM tabela_dados
WHERE
    id_member_anonymous IS NOT NULL
    AND text_content_anonymous IS NOT NULL
    AND TRIM(text_content_anonymous) <> ''
GROUP BY id_member_anonymous, CAST(date_message AS DATE), text_content_anonymous
HAVING COUNT(*) > 1
ORDER BY quantidade_envios DESC;
""").fetchdf()

display(qtd_mensagens_identicas)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,usuario,data,mensagem,quantidade_envios
0,c743967449a387ad2c1c7e03b2c45b36,2022-11-02,This community was blocked in Brazil following...,157
1,f3a914b5f81379cdf1689f202fe5ccd9,2022-10-06,Anal sex🙈,144
2,9b371a7d40877606e49d03281c8ba0f4,2022-10-06,Anal sex🙈,142
3,f233cf8b1d4ede06f32199a7e0081bf5,2022-11-03,This community was blocked in Brazil following...,142
4,f233cf8b1d4ede06f32199a7e0081bf5,2022-11-10,This community was blocked in Brazil following...,137
...,...,...,...,...
18859,dd9c0c1bfcb4e0e85548fea1b88401cd,2022-11-10,This community was blocked in Brazil following...,2
18860,8235541b3c7bb4d4b7621e92ae4d0511,2022-11-10,https://invierte.biz/archivos/15123,2
18861,c036e73cb7dae68e90f8fdc894ad0862,2022-11-10,Atentem-se ao item 9 da página 2 do relatório ...,2
18862,a7e85072244cae15446c9d517dc01a1a,2022-11-10,Temos pela frente uma data histórica 15 de Nov...,2


---

#### **20**. Mensagens idênticas compartilhadas pelo mesmo usuário em grupos distintos (e suas quantidades)

Essa consulta mostra quais mensagens foram compartilhadas pelo mesmo usuário em mais de um grupo. Isso ajuda a identificar comportamentos de distribuição ativa de conteúdo, como quando alguém 'espalha' uma mensagem por vários grupos. Filtrei só os casos com mais de um grupo distinto pra focar nos casos relevantes. O fluxo da consulta segue a estrutura abaixo:

- Agrupar por:
    - id_member_anonymous → quem enviou,
    - text_content_anonymous → o conteúdo da mensagem,
    - id_group_anonymous → o grupo onde foi enviada.

- Em seguida, recontar quantos grupos distintos receberam a mesma mensagem do mesmo usuário.

- Filtrar apenas os casos com mais de 1 grupo.



In [33]:
qtd_msg_identicas_grupos_distintos = conn.execute("""
    SELECT
        id_member_anonymous AS usuario,
        id_group_anonymous AS grupo,
        CAST(date_message AS DATE) AS data,
        text_content_anonymous AS mensagem,
        COUNT(*) AS quantidade_envios
    FROM tabela_dados
    WHERE
        id_member_anonymous IS NOT NULL
        AND id_group_anonymous IS NOT NULL
        AND text_content_anonymous IS NOT NULL
        AND TRIM(text_content_anonymous) <> ''
    GROUP BY
        id_member_anonymous, id_group_anonymous, CAST(date_message AS DATE), text_content_anonymous
    HAVING COUNT(*) > 1
    ORDER BY quantidade_envios DESC;
""").fetchdf()

display(qtd_msg_identicas_grupos_distintos)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,usuario,grupo,data,mensagem,quantidade_envios
0,c743967449a387ad2c1c7e03b2c45b36,f0944ed1832d8c1f1213d6123746f111,2022-11-02,This community was blocked in Brazil following...,157
1,f3a914b5f81379cdf1689f202fe5ccd9,89288ae64f9a7fcac9dc4613fac36185,2022-10-06,Anal sex🙈,144
2,9b371a7d40877606e49d03281c8ba0f4,89288ae64f9a7fcac9dc4613fac36185,2022-10-06,Anal sex🙈,142
3,f233cf8b1d4ede06f32199a7e0081bf5,857cd5311da1bdc15eb9e6918a47c6c6,2022-11-03,This community was blocked in Brazil following...,141
4,f233cf8b1d4ede06f32199a7e0081bf5,857cd5311da1bdc15eb9e6918a47c6c6,2022-11-04,This community was blocked in Brazil following...,137
...,...,...,...,...,...
7235,897d0b98691f2adc374db16ce6b15c0d,5b10d7739171149be6d9961e3350c071,2022-10-31,[31/10 02:35] Adilson Nadaline: Terminou agora...,2
7236,aaa62078e8a08c4923e20d534c6962d1,857cd5311da1bdc15eb9e6918a47c6c6,2022-10-31,📢 ATENÇÃO 🚨 🇧🇷🇧🇷🇧🇷\nNÃO DEPENDAM OU EXIJAM APE...,2
7237,c743967449a387ad2c1c7e03b2c45b36,f0944ed1832d8c1f1213d6123746f111,2022-10-31,"Alessandra, pressione o botão abaixo dentro de...",2
7238,b3ca12ce4b195df14996eed9845aee8b,857cd5311da1bdc15eb9e6918a47c6c6,2022-10-31,"Galera me chama Agnelo Junior ,sou Nordestino ...",2


---

#### **21**. Os 30 unigramas, bigramas e trigramas mais compartilhados

A ideia aqui é encontrar os 30 unigramas, bigramas e trigramas mais compartilhados nas mensagens. Para isso:

- Filtrei as mensagens, garantindo que só as não nulas e não vazias sejam analisadas.

- Tokenizei o texto, separando as mensagens em palavras com o scikit-learn. 

- A função `gerar_ngrams_sklearn` recebe a lista de mensagens e o número n para gerar os unigramas (1 palavra), bigramas (2 palavras) e trigramas (3 palavras). O CountVectorizer é utilizado para extrair essas sequências de palavras.

- Contei a frequência de cada n-grama e mostrei os 30 mais comuns. O `CountVectorizer` transforma as mensagens em uma matriz de contagem, e a frequência de cada n-grama é somada. Depois, os n-gramas são ordenados pela frequência em ordem decrescente.

In [34]:
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('fakeTelegram.BR_2022.csv')

# Filtrando mensagens válidas (não nulas e não vazias)
df_filtrado = df[
    df["text_content_anonymous"].notnull() &
    (df["text_content_anonymous"].str.strip() != "")
]

# Lista de mensagens
mensagens = df_filtrado["text_content_anonymous"].tolist()

# Função para gerar n-gramas
def gerar_ngrams_sklearn(textos, n):
    vectorizer = CountVectorizer(ngram_range=(n, n), stop_words='english')
    ngramas = vectorizer.fit_transform(textos)
    ngramas_freq = ngramas.sum(axis=0).A1  # Soma a frequência dos n-gramas
    ngramas_vocab = vectorizer.get_feature_names_out()  # Lista dos n-gramas
    ngramas_count = zip(ngramas_vocab, ngramas_freq)
    ngramas_count_sorted = sorted(ngramas_count, key=lambda x: x[1], reverse=True)
    return ngramas_count_sorted[:30]

# Gerando unigramas, bigramas e trigramas
top_unigramas = gerar_ngrams_sklearn(mensagens, 1)
top_bigramas = gerar_ngrams_sklearn(mensagens, 2)
top_trigramas = gerar_ngrams_sklearn(mensagens, 3)

# Exibindo os 30 mais comuns de cada tipo
print("🔹 Top 30 Unigramas:")
for unigrama, freq in top_unigramas:
    print(f"{unigrama}: {freq}")

print("\n🔹 Top 30 Bigramas:")
for bigrama, freq in top_bigramas:
    print(f"{bigrama}: {freq}")

print("\n🔹 Top 30 Trigramas:")
for trigram, freq in top_trigramas:
    print(f"{trigram}: {freq}")

KeyboardInterrupt: 

---

#### **22**. As 30 mensagens mais positivas (distintas)

Aqui eu filtrei as mensagens para considerar apenas aquelas com `score_sentiment` preenchido e texto válido. Depois, eliminei mensagens duplicadas (iguais em conteúdo) e ordenei do sentimento mais positivo para o menos. No final, selecionei as 30 mensagens distintas com os maiores scores de sentimento.

In [35]:
mensagens_mais_positivas = conn.execute("""
    WITH mensagens_unicas AS (
    SELECT
        text_content_anonymous,
        score_sentiment,
        ROW_NUMBER() OVER (PARTITION BY text_content_anonymous ORDER BY score_sentiment DESC) AS rn
    FROM tabela_dados
    WHERE
        text_content_anonymous IS NOT NULL
        AND TRIM(text_content_anonymous) <> ''
        AND score_sentiment IS NOT NULL
    )
    SELECT
        text_content_anonymous,
        score_sentiment
    FROM mensagens_unicas
    WHERE rn = 1
    ORDER BY score_sentiment DESC
    LIMIT 30;
""").fetchdf()

display(mensagens_mais_positivas)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,text_content_anonymous,score_sentiment
0,"Shemoneh Esreh – Amidá – A Grande Oração\n\n ,...",0.9992
1,"MENTIROSA!!!\n\n\nPensa direita, não é correto...",0.9990
2,"Pensa direita, não é correto compartilhar um v...",0.9990
3,"בונה ירושלים, Bo'ne Yetrushalayim - Benção da ...",0.9989
4,Os Sete Samurais \n \nQuando falamos sobre For...,0.9988
5,MANIFESTO À NAÇÃO BRASILEIRA\n\nEM DEFESA DO B...,0.9987
6,"Olá ,\n\nUM AMIGO ME PERGUNTOU EM QUEM EU VOTE...",0.9986
7,BOM DIA A TODOS. ENTENDO QUE ESTAMOS VIVENDO D...,0.9986
8,Uma forma bem legal\nPara quem ainda não sabe ...,0.9985
9,\nA Viva Esperança\n\nDepois das saudações in...,0.9985


---

#### **23**. As 30 mensagens mais negativas (distintas)

Filtrei as mensagens com `score_sentiment` válido e texto não vazio. Depois, removi duplicadas com base no conteúdo e ordenei da mais negativa para a menos negativa. Por fim, selecionei as 30 mensagens únicas com menor score de sentimento.

In [36]:
mensagens_mais_negativas = conn.execute("""
    WITH mensagens_unicas AS (
    SELECT
        text_content_anonymous,
        score_sentiment,
        ROW_NUMBER() OVER (PARTITION BY text_content_anonymous ORDER BY score_sentiment ASC) AS rn
    FROM tabela_dados
    WHERE
        text_content_anonymous IS NOT NULL
        AND TRIM(text_content_anonymous) <> ''
        AND score_sentiment IS NOT NULL
    )
    SELECT
        text_content_anonymous,
        score_sentiment
    FROM mensagens_unicas
    WHERE rn = 1
    ORDER BY score_sentiment ASC
    LIMIT 30;
""").fetchdf()

display(mensagens_mais_negativas)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,text_content_anonymous,score_sentiment
0,CAPITÃOZINHO DE MERDA 💩\nCAPITÃOZINHO DE MERDA...,-1.0000
1,ai ta na rua da um joinha 👍Quem ai ta na rua d...,-0.9999
2,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
3,ME CÚ NO PAU DO BOLSONARO \nME CÚ NO PAU DO BO...,-0.9999
4,"Gente, a Zambelli nos abandonou! Traiu a nossa...",-0.9999
5,"E nós aqui nos sacrificando, aguentando sol, c...",-0.9999
6,Intervenção é meu pau na tua mão\nIntervenção ...,-0.9999
7,fraude fraude fraude fraude fraude fraude frau...,-0.9999
8,ai ta na rua da um joinha 👍Quem ai ta na rua d...,-0.9998
9,na rua da um joinha 👍Quem ai ta na rua da um j...,-0.9998


---

#### **24**.  O usuário mais otimista

Aqui eu considerei apenas mensagens com `score_sentiment` válido e calculei a média de sentimento por usuário. Depois, selecionei o usuário com a maior média de sentimento entre todos — ou seja, aquele que mais compartilhou mensagens positivas em média.

In [37]:
user_otimista = conn.execute("""
    SELECT
    id_member_anonymous AS usuario,
    ROUND(AVG(score_sentiment), 3) AS media_sentimento,
    COUNT(*) AS total_mensagens
    FROM tabela_dados
    WHERE
        id_member_anonymous IS NOT NULL
        AND score_sentiment IS NOT NULL
    GROUP BY id_member_anonymous
    ORDER BY media_sentimento DESC
    LIMIT 1;
""").fetchdf()

display(user_otimista)

,usuario,media_sentimento,total_mensagens
0,1fc0ac9b3146dcb0d573f25db3d3ba55,0.995,1


---

#### **25**. O usuário mais pessimista

Igual à versão do otimista, mas agora com `ORDER BY ASC` para pegar quem tem a média mais baixa de sentimento. Também limita a apenas um resultado, ou seja, o mais pessimista.

In [40]:
user_pessimista = conn.execute("""
    SELECT
    id_member_anonymous AS usuario,
    ROUND(AVG(score_sentiment), 3) AS media_sentimento,
    COUNT(*) AS total_mensagens
    FROM tabela_dados
    WHERE
        id_member_anonymous IS NOT NULL
        AND score_sentiment IS NOT NULL
    GROUP BY id_member_anonymous
    ORDER BY media_sentimento ASC
    LIMIT 1;
""").fetchdf()

display(user_pessimista)

,usuario,media_sentimento,total_mensagens
0,243a43d802953032e1ea9a363546c3b8,-0.999,1


---

#### **26**. As 30 maiores mensagens

Filtrei apenas mensagens válidas (não nulas e não vazias), calculei o tamanho de cada uma com `LENGTH()` e ordenei da maior para a menor. No final, selecionei as 30 maiores mensagens em número de caracteres.

In [41]:
maiores_mensagens = conn.execute("""
    SELECT
    text_content_anonymous,
    LENGTH(text_content_anonymous) AS tamanho
    FROM tabela_dados
    WHERE text_content_anonymous IS NOT NULL
    AND TRIM(text_content_anonymous) <> ''
    ORDER BY tamanho DESC
    LIMIT 30;
""").fetchdf()

display(maiores_mensagens)

,text_content_anonymous,tamanho
0,Trecho do livro “Prática da Conspiração A Agen...,4096
1,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
2,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096
3,"1: Sansão foi a Gaza, e viu ali uma prostituta...",4096
4,PROPOSTAS DE BOLSONARO PARA 2023 - POSTADAS AG...,4096
5,CCCCCCCCCCCCCCCCCCCCCCCHHHHHHHHHHHHHHHHHHHHHOO...,4096
6,Trecho do livro “Prática da Conspiração A Agen...,4096
7,Trecho do livro “Prática da Conspiração A Agen...,4096
8,"1: Aconteceu também naqueles dias, quando não ...",4096
9,ROIA♘ ♞HERON★�CAVALO★§�DE★§�§TROIA♘ ♞HERON★�CA...,4096


---

#### **27**. As 30 menores mensagens

Filtrei apenas mensagens válidas, removi espaços em branco e ordenei pelo menor número de caracteres. Depois selecionei as 30 menores mensagens distintas.

In [42]:
menores_mensagens = conn.execute("""
    SELECT
    text_content_anonymous,
    LENGTH(text_content_anonymous) AS tamanho
    FROM tabela_dados
    WHERE text_content_anonymous IS NOT NULL
    AND TRIM(text_content_anonymous) <> ''
    ORDER BY tamanho ASC
    LIMIT 30;
""").fetchdf()

display(menores_mensagens)

,text_content_anonymous,tamanho
0,🤔,1
1,🤭,1
2,🤣,1
3,👀,1
4,🎯,1
5,😏,1
6,👇,1
7,🤣,1
8,😁,1
9,🙄,1


---

#### **28**. O dia em que foi publicado a maior quantidade de mensagens

Aqui eu agrupei as mensagens por data (`date_message`) e contei quantas mensagens foram publicadas em cada dia. Depois, selecionei o dia com o maior número de mensagens publicadas.

In [43]:
dia_maior_qtd_mensagem = conn.execute("""
    SELECT
    CAST(date_message AS DATE) AS dia,
    COUNT(*) AS total_mensagens
    FROM tabela_dados
    WHERE date_message IS NOT NULL
    GROUP BY dia
    ORDER BY total_mensagens DESC
    LIMIT 1;
""").fetchdf()

display(dia_maior_qtd_mensagem)

,dia,total_mensagens
0,2022-10-02,25243


---

#### **29**. As mensagens que possuem as palavras “FACÇÃO” e “CRIMINOSA”

Filtrei as mensagens que contêm ambas as palavras "FACÇÃO" e "CRIMINOSA" (sem distinção de maiúsculas/minúsculas). Utilizei o operador `LIKE` para encontrar essas palavras dentro do conteúdo das mensagens.

In [44]:
filtragem_por_palavra1 = conn.execute("""
    SELECT
    text_content_anonymous
    FROM tabela_dados
    WHERE text_content_anonymous IS NOT NULL
    AND LOWER(text_content_anonymous) LIKE '%facção%'
    AND LOWER(text_content_anonymous) LIKE '%criminosa%';
""").fetchdf()

display(filtragem_por_palavra1)

,text_content_anonymous
0,"Pensem!!!!!!'Se Houve fraude ,Nada foi feito ..."
1,#lulaladrãoseulugarénaprisão \n#PTFACÇÃOCRIMIN...
2,"Se depois de ler esse resumo do Plano do PT, v..."
3,"Se depois de ler esse resumo do Plano do PT, v..."
4,É exatamente isso que a facção criminosa CHAM...
...,...
134,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
135,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
136,😣🤯🤐 Alexandre de Moraes (Secretário-Geral do P...
137,Agressões covardes é termos uma facção crimino...


---

#### **30**. As mensagens que possuem a palavra “SEGURANÇA”

Filtrei as mensagens que contêm a palavra "SEGURANÇA", independentemente de maiúsculas ou minúsculas, utilizando o operador `LIKE` para localizar a palavra no conteúdo das mensagens.

In [45]:
filtragem_por_palavra2 = conn.execute("""
    SELECT
    text_content_anonymous
    FROM tabela_dados
    WHERE text_content_anonymous IS NOT NULL
    AND LOWER(text_content_anonymous) LIKE '%segurança%';
""").fetchdf()

display(filtragem_por_palavra2)

,text_content_anonymous
0,🌳 Mato Grosso intensifica ações contra crimes ...
1,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
2,⚡️🇧🇷 Actus Liberandi - Ordem De Cristo 🇧🇷\n\n✨...
3,No Telegram entrem em Configurações > Privacid...
4,No Telegram entrem em Configurações > Privacid...
...,...
2795,Profissão médica instada a dizer às mulheres a...
2796,"TSE, o relatório das Forças Armadas e o caso M..."
2797,"TSE, o relatório das Forças Armadas e o caso M..."
2798,✅ Notícias do Dia - Ano 6 n° 2183\nDia 11 de ...


---

Encerrando conexão com a base após todas as consultas:

In [46]:
conn.close()

## Fontes
- [DuckDB Documentation](https://duckdb.org/docs/stable/)
- [Convert csv to parquet file using python](https://stackoverflow.com/questions/50604133/convert-csv-to-parquet-file-using-python)
- [Extract hostname from a URL](https://stackoverflow.com/questions/13266534/extract-hostname-from-a-url)
- [Implement N-Grams using Python NLTK – A Step-By-Step Guide](https://www.askpython.com/python/examples/n-grams-python-nltk)